# Clase 6/10/20 - Video 13

# Material de clase
- Estudios: CabezaLd_Epilepsia - 1 & CT250MM_4

# Síntesis del Script

In [ ]:
escena = slicer.mrmlScene                                  # Se recupera la escena
volumen4D = escena.GetNodeByID('vtkMRMLMultiVolumeNode1')  # Se le solicita el estudio Multi-volumen a la escena
imagenvtk4D = volumen4D.GetImageData()                     # Se recupera el objeto ImageData() que  
                                                           # contiene los n estudios/volúmenes en el  tiempo

numero_frames = volumen4D.GetNumberOfFrames()              # Recuperamos el numero de frames (cantidad de estudios en != tiempo)
print('Frames: ' + str(numero_frames))                     # Cada estudio de los materiales, tiene alrededor de 70 Volúmenes

# Dimensiones
print(imagenvtk4D.GetBounds())                             # Obtenemos los rangos de las imagenes de los volumenes
                                                           # aprox:  [250 x 250 x 128 x 71]

# Filtro vtk para descomponer un volumen 4D
extract1 = vtk.vtkImageExtractComponents()                 # Se instancia el objeto de clase vtkImageExtractComponents()
extract1.SetInputData(imagenvtk4D)                         # y se le asigna la imagenvtk4D que contiene los volúmenes.

# Matrices de transformacion
ras2ijk = vtk.vtkMatrix4x4()                               # Se generan las matrices de transformacion vacias
ijk2ras = vtk.vtkMatrix4x4()
volumen4D.GetRASToIJKMatrix(ras2ijk)                       # Se solicita al estudio multivolumen sus matrices de Transf.
volumen4D.GetIJKToRASMatrix(ijk2ras)
volumenFijo = slicer.vtkMRMLScalarVolumeNode()             # Se crea un volumen nuevo que será el volumen fijo a registrar
volumenFijo.SetRASToIJKMatrix(ras2ijk)                     # Se asignan las Matrices de transformacion del volumen inicial
volumenFijo.SetIJKToRASMatrix(ijk2ras)

# Le asigno el volumen 3D fijo
imagen_fija = extract1.SetComponents(0)                    # Se solicita el Primer volumen extraido por el objeto extract1
extract1.Update()

volumenFijo.SetName('fijo')
volumenFijo.SetAndObserveImageData(extract1.GetOutput())   # El Primer volumen extraido, se asigna al volumen fijo
extract1.Update()
escena.AddNode(volumenFijo)                                # Se añade el volumen generado a la escena

# Extraigo la imagen movil
imagen_movil = extract1.SetComponents(50)                  # Seleccionamos un volumen lejano
extract1.Update()

# Creo un volumen movil, y realizamos el mismo procedimiento que con el fijo
volumenMovil = slicer.vtkMRMLScalarVolumeNode();           # Se crea el volumen movil
volumenMovil.SetRASToIJKMatrix(ras2ijk)                    # Se asignan las mismas matrices de transformacion del vol. original
volumenMovil.SetIJKToRASMatrix(ijk2ras)      
volumenMovil.SetAndObserveImageData(extract1.GetOutput())  # La imagen del volumen 50 extraido, se asigna al volumen movil
volumenMovil.SetName('movil')
escena.AddNode(volumenMovil)                               # Se añade a la escena el volumen movil

# Creamos la transformada para alinear los volumenes
transformadaSalida = slicer.vtkMRMLLinearTransformNode()   # Se instancia un objeto de la clase vtkMRMLLinearTransformNode()
transformadaSalida.SetName('Transformada de registro')     # Se nombra
slicer.mrmlScene.AddNode(transformadaSalida)               # y se añade a la escena como un Nodo
            
# Parametros para la operacion de registro
parameters = {}
#parameters['InitialTransform'] = transI.GetID()           # Por si hay una transformación inicial
parameters['fixedVolume'] = volumenFijo.GetID()
parameters['movingVolume'] = volumenMovil.GetID()
parameters['transformType'] = 'Rigid'
parameters['outputTransform'] = transformadaSalida.GetID()

cliNode = slicer.cli.run(slicer.modules.brainsfit,None,parameters, wait_for_completion=True)


# Script Guiado

### Adquisicion

In [ ]:
escena = slicer.mrmlScene                                  # Se recupera la escena
volumen4D = escena.GetNodeByID('vtkMRMLMultiVolumeNode1')  # Se le solicita el estudio Multi-volumen a la escena
imagenvtk4D = volumen4D.GetImageData()                     # Se recupera el objeto ImageData() que  
                                                           # contiene los n estudios/volúmenes en el  tiempo

numero_imagenes = volumen4D.GetNumberOfFrames()            # Recuperamos el numero de volúmenes o estudios
print('imagenes: ' + str(numero_imagenes))                 # Cada estudio de los materiales, tiene alrededor de 70 Volúmenes

# Dimensiones
print(imagenvtk4D.GetBounds())                             # Obtenemos los rangos de las imagenes de los volumenes
                                                           # aprox:  [250 x 250 x 128 x 71]

### Descomposicion del Estudio 4D usando vtk.vtkImageExtractComponents()

In [ ]:
# Filtro vtk para descomponer un volumen 4D
extract1 = vtk.vtkImageExtractComponents()                 # Se instancia el objeto de clase vtkImageExtractComponents()
extract1.SetInputData(imagenvtk4D)                         # y se le asigna la imagenvtk4D que contiene los volúmenes.

# Matrices de transformacion varias
ras2ijk = vtk.vtkMatrix4x4()                               # Se generan las matrices de transformacion vacias
ijk2ras = vtk.vtkMatrix4x4()
volumen4D.GetRASToIJKMatrix(ras2ijk)                       # Se solicita al estudio multivolumen sus matrices de Transf.
volumen4D.GetIJKToRASMatrix(ijk2ras)
volumenFijo = slicer.vtkMRMLScalarVolumeNode()             # Se crea un volumen nuevo que será el volumen fijo a registrar
volumenFijo.SetRASToIJKMatrix(ras2ijk)                     # Se asignan las Matrices de transformacion del volumen inicial
volumenFijo.SetIJKToRASMatrix(ijk2ras)

# Le asigno el volumen 3D fijo
imagen_fija = extract1.SetComponents(0)                    # Se solicita el Primer volumen extraido por el objeto extract1
extract1.Update()

volumenFijo.SetName('fijo')
volumenFijo.SetAndObserveImageData(extract1.GetOutput())   # La imagen del Primer volumen extraido, se asigna al volumen fijo
extract1.Update()
escena.AddNode(volumenFijo)                                # Se añade el volumen generado a la escena

# Extraigo la imagen movil
imagen_movil = extract1.SetComponents(50)                  # Seleccionamos un volumen lejano
extract1.Update()

# Creo un volumen movil, y realizamos el mismo procedimiento que con el fijo
volumenMovil = slicer.vtkMRMLScalarVolumeNode();           # Se crea el volumen movil
volumenMovil.SetRASToIJKMatrix(ras2ijk)                    # Se asignan las mismas matrices de transformacion del vol. original
volumenMovil.SetIJKToRASMatrix(ijk2ras)      
volumenMovil.SetAndObserveImageData(extract1.GetOutput())  # La imagen del volumen 50 extraido, se asigna al volumen movil
volumenMovil.SetName('movil')
escena.AddNode(volumenMovil) 

### Se crea la matriz de transformacion que usará el módulo de Registro (BRAINS)

In [ ]:
# Creamos la transformada para alinear los volumenes
transformadaSalida = slicer.vtkMRMLLinearTransformNode()   # Se instancia un objeto de la clase vtkMRMLLinearTransformNode()
transformadaSalida.SetName('Transformada de registro')     # Se nombra
slicer.mrmlScene.AddNode(transformadaSalida)               # y se añade a la escena como un Nodo

### Se Recopilan los parámetros y se ejecuta el Módulo Brains usando Command Line Interface

In [ ]:
# Parametros para la operacion de registro
parameters = {}
#parameters['InitialTransform'] = transI.GetID()           # Por si hay una transformación inicial
parameters['fixedVolume'] = volumenFijo.GetID()
parameters['movingVolume'] = volumenMovil.GetID()
parameters['transformType'] = 'Rigid'
parameters['outputTransform'] = transformadaSalida.GetID()

cliNode = slicer.cli.run(slicer.modules.brainsfit,None,parameters, wait_for_completion=True)